In [1]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
data = pd.read_csv('../data/all_haiku.csv')

data = data.iloc[0:1000]

data['haiku'] = data['0']+' '+data['1']+' '+data['2']

data['haiku'] = data['haiku'].apply(lambda x: str(x).strip().replace('-','').lower())


In [3]:
#Leer los datos

text = ''

for i in data['haiku']:
    text += i + ' '

#vocab = sorted(set(text))
vocab = sorted(set(list(filter(lambda x: x != "", text.split(" ")))))

print(f'Text is {len(text)} long')
print(f'Text has {len(vocab)} unique characters')


Text is 50931 long
Text has 2728 unique characters


In [4]:
#Procesamiento del texto

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

txt_words = list(filter(lambda x: x != "", text.split(" ")))

text_as_int = np.array([char2idx[c] for c in txt_words])


In [5]:
seq_length = 100
examples_per_epoch = len(txt_words)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

fishing
boats
colors
of
the


In [6]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))


"fishingboatscolorsoftherainbowashwednesdaytryingtoremembermydreamsnowymornpouringanothercupofblackcoffeeshortestdayflamesdanceintheovenhazehalfthehorsehiddenbehindthehouselowsuntheladyinredonhighheelsadventthepassingstrangerfartstarnabubbleintheicesnowflakesnewasphaltintheholescrystalnight'gustsofrainoutsiderainthesoundofahorsegallopingthroughleaveswinterstarssuddenlyawhiffofperfumehungryhalfofthemoonhiddenrainanotherleafdownshariathesoundofonehandclapping"
"thesoundofgeesedrownedbythesoundofthetrainthismorningautumnsunmyshadowovertombstonesflyfishing;thesoundofthewindinthereeldecemberalongshadowjoinsanotherendofpathsnowflakesmeltingonthepondmorningfrostsheleavesfirsteveningwalksmelloftarbetweenpinesdachauablueskyabovethechimneysdeepautumn;theapplecolderinthetree.visitingthegravesstrongertheoctoberwindatmygrandparents'overthehedgeadragonflyeastabubbleburstsonsurfacefullmoonrainthewhitelilaclowmyhandon"
"herhipfullmoonthreepetalsfallfromthepurpleconeflower...almostsummerinstantmessagem

In [7]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))
    

Input data:  "fishingboatscolorsoftherainbowashwednesdaytryingtoremembermydreamsnowymornpouringanothercupofblackcoffeeshortestdayflamesdanceintheovenhazehalfthehorsehiddenbehindthehouselowsuntheladyinredonhighheelsadventthepassingstrangerfartstarnabubbleintheicesnowflakesnewasphaltintheholescrystalnight'gustsofrainoutsiderainthesoundofahorsegallopingthroughleaveswinterstarssuddenlyawhiffofperfumehungryhalfofthemoonhiddenrainanotherleafdownshariathesoundofonehand"
Target data:  "boatscolorsoftherainbowashwednesdaytryingtoremembermydreamsnowymornpouringanothercupofblackcoffeeshortestdayflamesdanceintheovenhazehalfthehorsehiddenbehindthehouselowsuntheladyinredonhighheelsadventthepassingstrangerfartstarnabubbleintheicesnowflakesnewasphaltintheholescrystalnight'gustsofrainoutsiderainthesoundofahorsegallopingthroughleaveswinterstarssuddenlyawhiffofperfumehungryhalfofthemoonhiddenrainanotherleafdownshariathesoundofonehandclapping"


In [8]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print(f'Step     {i}')
    print(f'   input: {int(input_idx), repr(idx2char[input_idx])}')
    print(f'   expected output: {int(target_idx), repr(idx2char[target_idx])}')

Step     0
   input: (891, "'fishing'")
   expected output: (257, "'boats'")
Step     1
   input: (257, "'boats'")
   expected output: (516, "'colors'")
Step     2
   input: (516, "'colors'")
   expected output: (1643, "'of'")
Step     3
   input: (1643, "'of'")
   expected output: (2405, "'the'")
Step     4
   input: (2405, "'the'")
   expected output: (1879, "'rainbow'")


In [9]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [10]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [11]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [12]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [13]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 2728) # (batch_size, sequence_length, vocab_size)


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           698368    
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 2728)          2796200   
Total params: 7,432,872
Trainable params: 7,432,872
Non-trainable params: 0
_________________________________________________________________


In [15]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()


In [16]:
sampled_indices

array([2535, 2386, 2724,  497,  433, 2467, 1735, 2119, 2280, 1937, 2147,
        848,  263, 2726,  408, 2180, 1395, 1012,   41,  565, 1807,  320,
        949, 2695,   37,  704, 1544, 1405,   91,  166, 2249, 1607,  913,
        507, 2228, 1539,  271, 1240, 1423, 2578,   54, 2700, 1606, 2568,
       1858,  886, 1839, 1758, 2569, 2584,  514, 1338,  324, 2453,  469,
        785, 2309, 1177,  976,  137,  505, 1366, 1595, 2241, 1206, 1747,
       2705, 1551, 1264, 1699,  728,  450, 1699,  613, 1186,  123, 1372,
        959,  583,  144, 2568,  189, 1126,  917, 1163, 1580,  144,  908,
       1888, 1526,  105, 1252, 2175, 1470,  852, 1120,  943,  473, 1762,
       1179])

In [17]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))


Input: 
 'pollentwinbedwearingyourflannelshirtkeepsmewarmbackporchlaughterechoesbetweenclapsofthunderendofsummerstrandsofanabandonedkiteflutterinthewindinthehospiceanoutoftunepianohelpshersingwithoutwordshandinhandwhitepetalsblossomaswepassabandonedhomeanoldlifelaidtorestamongweedsandtreeshumidmorningthescentoflilacsfillseachbreathicypavementunopenedbudsshrinkandfallfromthetreesbloodmoonforcedtobecomeawomanlightrainlavenderblossominginasketchbookrainpuddlea'

Next Char Predictions: 
 "unknottingtears~cluecherrytouchpaymentsightsteamresumeskinfarmhousebook—cemeterysmileslostglideadvancecow'spointsbuddhaforkwriteadjustdove'smothslowerantiquebarkspraynightfall..floatscoffee.sparrowmothbottleinsomniamailwalksage?x'snight'svotepurplefireflies.pregnantpeppersvoyageurswar.collectedleftoverbugtissueclayenclosedstreetlightshonkingfrostedautumn'scocklisteningnestspinnedhurricanepenyearsmournersi’vepajamadripschinookpajamacuttinghospitalasksloadedfreecritiquesawhilevotebedtimehearseflourhisnature

In [18]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (64, 100, 2728)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       7.9114013


In [19]:
model.compile(optimizer='adam', loss=loss)


In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [21]:
#Training with 30 EPOCHS

EPOCHS=30

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/30
1/1 [==============================] - 2s 2s/step - loss: 7.9114
Epoch 2/30
1/1 [==============================] - 2s 2s/step - loss: 7.9035
Epoch 3/30
1/1 [==============================] - 1s 1s/step - loss: 7.8925
Epoch 4/30
1/1 [==============================] - 1s 1s/step - loss: 7.8672
Epoch 5/30
1/1 [==============================] - 2s 2s/step - loss: 7.7260
Epoch 6/30
1/1 [==============================] - 1s 1s/step - loss: 9.0242
Epoch 7/30
1/1 [==============================] - 2s 2s/step - loss: 7.1213
Epoch 8/30
1/1 [==============================] - 1s 1s/step - loss: 7.3540
Epoch 9/30
1/1 [==============================] - 1s 1s/step - loss: 7.0855
Epoch 10/30
1/1 [==============================] - 1s 1s/step - loss: 6.7798
Epoch 11/30
1/1 [==============================] - 0s 187ms/step - loss: 6.8119
Epoch 12/30
1/1 [==============================] - 0s 434ms/step - loss: 6.8695
Epoch 13/30
1/1 [==============================] - 0s 164ms/step - loss: 6.7635

In [22]:
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints/ckpt_30'

In [23]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))


In [24]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            698368    
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 2728)           2796200   
Total params: 7,432,872
Trainable params: 7,432,872
Non-trainable params: 0
_________________________________________________________________


In [25]:
def generate_text(model, start_list):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_list]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (text_generated)

In [26]:
print(generate_text(model, start_list=['fishing', 'boats', 'colors', 'of', 'the', 'rainbow']))


['too', 'drawn', 'hour', 'cock', 'schoolgirls', 'halfway', 'spring', 'home', 'on', 'i', 'one', 'watching', 'argument', 'left', 'black', 'farm', 'spring', 'an', 'oak', 'waiting', 'vanishes', 'just', 'changing', '...', 'sound', 'tombstones', '~', 'father', 'bloom', 'smile', 'train', 'young', 'bus', 'house', 'orgasm', 'bottle', 'moon', 'across', 'buried', 'stars', 'between', 'is', 'grave', 'autumn', 'last', 'the', 'stone', 'where', 'of', 'hospice', 'that', 'patio', 'desert', 'he', 'hot', 'the', 'on', 'night', 'huddle', 'full', 'pool', 'through', 'moon', 'starless', 'elder—', 'and', 'of', 'after', 'the', 'watching', 'of', 'by', 'scandal', 'and', 'car', 'of', 'dried', 'chill', 'breakfast', 'fork', 'winds', 'bus', 'train', 'a', 'another', 'long', 'year—', 'years', 'glides', 'life', 'the', 'avalanche', 'at', 'the', 'wait', 'along', 'buns', 'that', 'footprints', 'scramble', 'cracks', 'more', 'the', 'words', 'cereus', 'a', 'many', 'of', 'wind', 'birds', 'night', 'in', 'steps', 'shrink', 'betwee